# Libraries

In [1]:
import sqlite3
import numpy as np
from collections import OrderedDict, namedtuple
from distutils.spawn import find_executable
import matplotlib.pyplot as plt

import sys
import subprocess
import pathlib
import tempfile
import importlib
import os
import time
import random
import pickle

import math

from configparser import ConfigParser

# from google.protobuf.json_format import MessageToDict

In [2]:
# DEVICE = 0

import os
import time
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import torch
torch.backends.cudnn.benchmark = True
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data

# torch.cuda.set_device(DEVICE)
torch.cuda.empty_cache() 
torch.backends.cudnn.benchmark=True

In [3]:
np.random.seed(0)
torch.manual_seed(0)


import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

# load the dataset

In [4]:
with open('scrimmage5_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [5]:
train_data = link_dataset[:400] 
test_data = link_dataset[400:]

In [6]:
train_x = torch.cat(tuple(link[0] for link in train_data),dim=0)
train_y = torch.cat(tuple(link[1] for link in train_data),dim=0)

test_x = torch.cat(tuple(link[0] for link in test_data),dim=0)
test_y = torch.cat(tuple(link[1] for link in test_data),dim=0)

In [7]:
test_x.size()

torch.Size([4062650, 20])

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(20, 128)
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 128)
        self.linear4 = nn.Linear(128, 128)
        self.linear5 = nn.Linear(128, 2)
        
        self.relu = nn.ReLU()
        
#         self.dropout = nn.Dropout(0.6)

    
    def forward(self, input):

        output = self.linear1(input)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear2(output)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear3(output)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear4(output)
        output = self.relu(output)
#         output = self.dropout(output)

        output = self.linear5(output)
        
        return output

# Train Model

In [9]:
NUM_EPOCHS = 10
BATCH_SIZE = 1024

model = MLP()
model.cuda()

loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001)

train_dataloader = data.DataLoader(
    dataset=data.TensorDataset(train_x,train_y), 
    batch_size=BATCH_SIZE, shuffle=True, num_workers=16, pin_memory=True)

test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

model.train()
for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        test_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

#     print(f'train loss:{train_loss.item()/train_size: .5f}, '
#           f'test loss:{test_loss.item()/test_size: .5f}, '
#           f'overall accuracy:{(predict == target).sum() / target.size}, '
#           f'positive accuracy:{predict[target==1].sum() / target.sum()}, '
#           f'negative accuracy:{(np.logical_not(target).sum() - predict[np.logical_not(target)].sum()) / np.logical_not(target).sum()}')

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'test loss:{test_loss.item()/test_size: .5f}')

Epoch 1/10, Validation:   0%|                                  | 0/3968 [00:00<?, ?it/s]

train loss: 0.42545, test loss: 0.61566


Epoch 2/10, Validation:   0%|                                  | 0/3968 [00:00<?, ?it/s]

train loss: 0.40070, test loss: 0.62686


Epoch 3/10, Validation:   0%|                                  | 0/3968 [00:00<?, ?it/s]

...

train loss: 0.38781, test loss: 0.61913


Epoch 9/10, Validation:   0%|                                  | 0/3968 [00:00<?, ?it/s]

train loss: 0.38722, test loss: 0.62051


Epoch 9/10, Validation: 100%|██████████████████████| 3968/3968 [00:28<00:00, 140.20it/s]

train loss: 0.38670, test loss: 0.62581


In [10]:
test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

predict = []
target = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)

tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()

print(f'True Positive:{tp}, '
      f'True Negative:{tn}, '
      f'False Positive:{fp}, '
      f'False Negative:{fn}')

True Positive:2503419, True Negative:324669, False Positive:796175, False Negative:438387


In [11]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.6961190356048392, Positive Accuracy:0.7587051619077984, Negative Accuracy:0.42548515443165374, Weighted Accuracy:0.592095158169726


# Train Model 1 Epoch

In [12]:
NUM_EPOCHS = 1
BATCH_SIZE = 1024

model = MLP()
model.cuda()

loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.001)

train_dataloader = data.DataLoader(
    dataset=data.TensorDataset(train_x,train_y), 
    batch_size=BATCH_SIZE, shuffle=True, num_workers=16, pin_memory=True)

test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

model.train()
for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        test_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

#     print(f'train loss:{train_loss.item()/train_size: .5f}, '
#           f'test loss:{test_loss.item()/test_size: .5f}, '
#           f'overall accuracy:{(predict == target).sum() / target.size}, '
#           f'positive accuracy:{predict[target==1].sum() / target.sum()}, '
#           f'negative accuracy:{(np.logical_not(target).sum() - predict[np.logical_not(target)].sum()) / np.logical_not(target).sum()}')

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'test loss:{test_loss.item()/test_size: .5f}')

Epoch 0/1, Validation: 100%|███████████████████████| 3968/3968 [00:28<00:00, 139.26it/s]

train loss: 0.42571, test loss: 0.60829


# Test

In [13]:
test_dataloader = data.DataLoader(
    data.TensorDataset(test_x,test_y), 
    batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

predict = []
target = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()

print(f'True Positive:{predict[target==1].sum()}, '
      f'True Negative:{(target==0).sum() - predict[target==0].sum()}, '
      f'False Positive:{predict[target==0].sum()}, '
      f'False Negative:{(target==1).sum() - predict[target==1].sum()}')
print(f'True Positive:{predict[target==1].sum()}, '
      f'True Negative:{(target==0).sum() - predict[target==0].sum()}, '
      f'False Positive:{predict[target==0].sum()}, '
      f'False Negative:{(target==1).sum() - predict[target==1].sum()}')

True Positive:2508189, True Negative:345989, False Positive:774855, False Negative:433617
True Positive:2508189, True Negative:345989, False Positive:774855, False Negative:433617


In [14]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.7025409523340677, Positive Accuracy:0.7639827550285649, Negative Accuracy:0.44379981683055286, Weighted Accuracy:0.6038912859295589


# Bagging 9

In [15]:
with open('scrimmage5_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [16]:
NUM_EPOCHS = 1
model_list = []
BATCH_SIZE = 1024

for model_idx in range(9):
    
    train_data = []
    for link_idx in np.random.choice(200,size=200,replace=True):
        train_data.append(link_dataset[link_idx])
    test_data = link_dataset[200:]
    
    train_x = torch.cat(tuple(link[0] for link in train_data),dim=0)
    train_y = torch.cat(tuple(link[1] for link in train_data),dim=0)
    test_x = torch.cat(tuple(link[0] for link in test_data),dim=0)
    test_y = torch.cat(tuple(link[1] for link in test_data),dim=0)

    model = MLP()
    model.cuda()

    loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
    optimizer = optim.Adam(model.parameters(), 0.001)

    train_dataloader = data.DataLoader(
        dataset=data.TensorDataset(train_x,train_y), 
        batch_size=BATCH_SIZE, shuffle=True, num_workers=16, pin_memory=True)

    test_dataloader = data.DataLoader(
        data.TensorDataset(test_x,test_y), 
        batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

    model.train()
    for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

        progress_training_epoch = tqdm(
            train_dataloader, 
            desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
            miniters=1, ncols=88, position=0,
            leave=True, total=len(train_dataloader), smoothing=.9)

        progress_validation_epoch = tqdm(
            test_dataloader, 
            desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
            miniters=1, ncols=88, position=0, 
            leave=True, total=len(test_dataloader), smoothing=.9)

        train_loss = 0
        train_size = 0
        for idx, (input, target) in enumerate(progress_training_epoch):
            input = input.cuda()
            target = target.cuda()
            model.zero_grad()
            predict = model(input)
            loss = loss_function(predict, target)
            loss.backward()
            optimizer.step()
            train_loss += loss * target.size()[0]
            train_size += target.size()[0]

        test_loss = 0
        test_size = 0
        model.eval()
        with torch.no_grad():
            for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
                batch_input = batch_input.cuda()
                batch_target = batch_target.cuda()
                batch_predict = model(batch_input)
                loss = loss_function(batch_predict, batch_target)
                test_loss += loss * batch_target.size()[0]
                test_size += batch_target.size()[0]

        print(f'train loss:{train_loss.item()/train_size: .5f}, '
              f'test loss:{test_loss.item()/test_size: .5f}')
        
    model_list.append(model) 

Epoch 0/1, Validation: 100%|███████████████████████| 7810/7810 [00:23<00:00, 337.67it/s]


train loss: 0.39365, test loss: 0.63110


Epoch 0/1, Validation: 100%|███████████████████████| 7810/7810 [00:23<00:00, 326.98it/s]


train loss: 0.38089, test loss: 0.62694


Epoch 0/1, Validation: 100%|███████████████████████| 7810/7810 [00:23<00:00, 331.80it/s]


...

train loss: 0.38050, test loss: 0.61537


Epoch 0/1, Validation: 100%|███████████████████████| 7810/7810 [00:24<00:00, 312.73it/s]


train loss: 0.41071, test loss: 0.63033


Epoch 0/1, Validation: 100%|███████████████████████| 7810/7810 [00:24<00:00, 316.79it/s]

train loss: 0.41027, test loss: 0.62945


# Test

In [17]:
predict_agg = []
target_agg = []

for model in model_list:
    
    test_dataloader = data.DataLoader(
        data.TensorDataset(test_x,test_y), 
        batch_size=BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

    progress_validation_epoch = tqdm(
        test_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)
    
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
    
    predict_agg.append(predict)
    target_agg.append(target)

predict = (np.array(predict_agg).sum(axis=0) > 1.5)
target = target_agg[0]
tp = predict[target==1].sum()
tn = (target==0).sum() - predict[target==0].sum()
fp = predict[target==0].sum()
fn = (target==1).sum() - predict[target==1].sum()
print(f'True Positive:{tp}, '
      f'True Negative:{tn}, '
      f'False Positive:{fp}, '
      f'False Negative:{fn}')
#print(f'True Positive:{predict[target==1].sum()}, '
      #f'True Negative:{(target==0).sum() - predict[target==0].sum()}, '
      #f'False Positive:{predict[target==0].sum()}, '
      #f'False Negative:{(target==1).sum() - predict[target==1].sum()}')

Epoch 0/1, Validation: 100%|███████████████████████| 7810/7810 [00:12<00:00, 646.67it/s]


True Positive:5613242, True Negative:147174, False Positive:2156124, False Negative:80398


In [18]:
accr = (tp+tn)/(tp+tn+fp+fn)
posaccr = tp/(tp+fp)
negaccr = tn/(tn+fn)
print(f'Accuracy:{accr}, '
      f'Positive Accuracy:{posaccr}, '
      f'Negative Accuracy:{negaccr}, '
      f'Weighted Accuracy:{0.5*(posaccr+negaccr)}')

Accuracy:0.7203277054292531, Positive Accuracy:0.7224839195373213, Negative Accuracy:0.6467140069955882, Weighted Accuracy:0.6845989632664548
